### Import Required Libraries and Set Up Environment Variables

In [1]:
# Dependencies
import requests
import time
from dotenv import load_dotenv
import os
import pandas as pd
import json
import matplotlib as plt
import re

In [2]:
# Set environment variables from the .env in the local environment
load_dotenv()

nyt_api_key = os.getenv("NYT_API_KEY")
tmdb_api_key = os.getenv("TMDB_API_KEY")

In [3]:
collected_data = "output\collected_data.csv"
collected_data = pd.read_csv(collected_data)

In [4]:
selected_data = pd.concat([collected_data.iloc[0:2], collected_data.iloc[3:31]])

### Access the New York Times API

In [6]:
# Set the base URL
url = "https://api.nytimes.com/svc/search/v2/articlesearch.json"

# Filter for movie reviews with "love" in the headline
# section_name should be "Movies"
# type_of_material should be "Review"
filter_query = 'section_name:"Movies" AND type_of_material:"Review" AND headline:"love"'

# Use a sort filter, sort by newest
sort = "newest"

# Select the following fields to return:
# headline, web_url, snippet, source, keywords, pub_date, byline, word_count
field_list = "headline,web_url,snippet,source,keywords,pub_date,byline,word_count"

# Search for reviews published between a begin and end date
begin_date = "20130101"
end_date = "20230531"

# Build URL
params = {
    'q': filter_query,
    'sort': sort,
    'fl': field_list,
    'begin_date': begin_date,
    'end_date': end_date,
}

response = requests.get(url, params=params)
if response.status_code == 200:
    data = response.json()
    print(data)
else:
    print(f"Error: {response.status_code}")

# Print the URL for verification
print(response.url)

Error: 401
https://api.nytimes.com/svc/search/v2/articlesearch.json?q=section_name%3A%22Movies%22+AND+type_of_material%3A%22Review%22+AND+headline%3A%22love%22&sort=newest&fl=headline%2Cweb_url%2Csnippet%2Csource%2Ckeywords%2Cpub_date%2Cbyline%2Cword_count&begin_date=20130101&end_date=20230531


In [7]:

reviews = []
def get_reviews(page):
    url = "https://api.nytimes.com/svc/search/v2/articlesearch.json"
    params = {
        'api-key': nyt_api_key,
        'q': filter_query,
        'sort': sort,
        'fl': field_list,
        'begin_date': begin_date,
        'end_date': end_date,
        'page': page
    }
    response = requests.get(url, params=params)
    if response.status_code == 200:
        data = response.json()
        return data['response']['docs']
    else:
        print(f"Error: {response.status_code}")
        return []

for page in range(20):
    page_reviews = get_reviews(page)
    reviews.extend(page_reviews)
    time.sleep(1) 

print(f"Total reviews fetched: {len(reviews)}")
reviews_df = pd.DataFrame(reviews)
print(reviews_df.columns)
if 'headline' in reviews_df.columns:
    reviews_df['title'] = reviews_df['headline'].apply(lambda x: x.get('main', ''))
if 'byline' in reviews_df.columns:
    reviews_df['byline'] = reviews_df['byline'].apply(lambda x: x.get('person', [{}])[0].get('firstname', '') + ' ' + x.get('person', [{}])[0].get('lastname', ''))



Error: 429
Error: 429
Error: 429
Error: 429
Error: 429
Error: 429
Error: 429
Error: 429
Error: 429
Error: 429
Total reviews fetched: 0
RangeIndex(start=0, stop=0, step=1)


In [8]:
# Create an empty list to store the reviews
reviews = []

# loop through pages 0-19
def get_reviews(page):
 for page in range(20):
    page_reviews = get_reviews(page)
    reviews.extend(page_reviews)
    print(f"Collected {len(reviews)} Reviews:", reviews)
    # create query with a page number
    # API results show 10 articles at a time

    base_url = "nyt_api_key"
    query = f"{base_url}?page={page}"
    def get_reviews(page):
       return [f"Review {i + page * 10}" for i in range(10)]
     
    # Make a "GET" request and retrieve the JSON
    def get_reviews(page):
     if response.status_code == 200:
        data = response.json()
        return data['articles']
     else:
        print(f"Failed to retrieve data for page {page}")
        return []

    # Add a twelve second interval between queries to stay within API query limits
time.sleep(12)
    
    # Try and save the reviews to the reviews_list
reviews_list = []
        # loop through the reviews["response"]["docs"] and append each review to the list
def page_reviews(page):  
 for review in page_reviews:
    reviews_list.append(review)
    if not page_reviews:
        print(f"No results found for page {page}")
        break
        # Print the page that was just retrieved
print(f"Collected {len(reviews_list)} Reviews:", reviews_list)

        # Print the page number that had no results then break from the loop

Collected 0 Reviews: []


In [9]:
reviews_df = pd.DataFrame(reviews)
print(reviews_df.columns)

RangeIndex(start=0, stop=0, step=1)


In [10]:
# Preview the first 5 results in JSON format
# Use json.dumps with argument indent=4 to format data
print("First 5 reviews in JSON format:")
print(json.dumps(reviews_list[:5], indent=2))

First 5 reviews in JSON format:
[]


In [11]:
# Convert reviews_list to a Pandas DataFrame using json_normalize()
reviews_df = pd.json_normalize(reviews_list)
print("DataFrame preview:")
print(reviews_df.head())


DataFrame preview:
Empty DataFrame
Columns: []
Index: []


In [12]:
print("First 5 reviews in JSON format:")
print(json.dumps(reviews[:5], indent=4))

First 5 reviews in JSON format:
[]


In [13]:
# Extract the title from the "headline.main" column and
# save it to a new column "title"
# Title is between unicode characters \u2018 and \u2019. 
# End string should include " Review" to avoid cutting title early
title = {"x"}
def extract_title(headline):
    match = re.search(r'\u2018(.*?)\u2019 Review', headline)
    return match.group(1) if match else None
if 'headline.main' in reviews_df.columns:
    reviews_df['title'] = reviews_df['headline.main'].apply(extract_title)
else:
    print("Column 'headline.main' not found in the DataFrame")
reviews_df.to_csv('nyt_movie_reviews.csv', index=False)
print(reviews_df.head())
    

Column 'headline.main' not found in the DataFrame
Empty DataFrame
Columns: []
Index: []


In [14]:
# Extract 'name' and 'value' from items in "keywords" column
def extract_keywords(keyword_list):
    extracted_keywords = ""
    for item in keyword_list:
        keyword = f"{item['name']}: {item['value']};" 
        # Append the keyword item to the extracted_keywords list
        extracted_keywords += keyword
    return extracted_keywords

# Fix the "keywords" column by converting cells from a list to a string
if 'keywords' in reviews_df.columns:
    reviews_df['keywords'] = reviews_df['keywords'].apply(extract_keywords)
else:
    print("Column 'keywords' not found in the DataFrame")
reviews_df.to_csv('nyt_movie_reviews.csv', index=False)
print(reviews_df.head())

Column 'keywords' not found in the DataFrame
Empty DataFrame
Columns: []
Index: []


In [15]:
# Create a list from the "title" column using to_list()
# These titles will be used in the query for The Movie Database
if 'title' in reviews_df.columns:
    titles_list = reviews_df['title'].to_list()
    print("List of titles:")
    print(titles_list)
else:
    print("Column 'title' not found in the DataFrame")

Column 'title' not found in the DataFrame


### Access The Movie Database API

In [16]:
# Create an empty list to store the results

# Create a request counter to sleep the requests after a multiple
# of 50 requests

# Loop through the titles

    # Add 1 to the request counter
    
    # Perform a "GET" request for The Movie Database

    # Include a try clause to search for the full movie details.
    # Use the except clause to print out a statement if a movie
    # is not found.


# Get movie id

    
# Make a request for a the full movie details


# Execute "GET" request with url

# Extract the genre names into a list


# Extract spoken language names into a list

# Extract production country names into a list

# Add the relevant data to a dictionary and
# append it to the tmdb_movies_list list

        
# Print out the title that was found

# Function to fetch movie ID from TMDB


# Function to fetch full movie details from TMDB using movie ID


# Initialize list to store full movie details


# Loop through the titles and fetch details


# Add the relevant data to a dictionary and append it to the tmdb_movies_list list
  


# Convert the tmdb_movies_list to a DataFrame
API_KEY = 'tmdb_api_key'
def fetch_movie_id(title, api_key):
    search_url = f'https://api.themoviedb.org/3/search/movie?api_key={api_key}&query={title}'
    response = requests.get(search_url)
    if response.status_code == 200:
        data = response.json()
        if data['results']:
            return data['results'][0]['id']
    return None
def fetch_movie_details(movie_id, api_key):
    details_url = f'https://api.themoviedb.org/3/movie/{movie_id}?api_key={api_key}'
    response = requests.get(details_url)
    if response.status_code == 200:
        return response.json()
    return None
tmdb_movies_list = []
collected_data_path = 'output/collected_data.csv'
collected_data = pd.read_csv(collected_data_path)
selected_data = pd.concat([collected_data.iloc[0:2], collected_data.iloc[3:31]])

request_counter = 0

for title in selected_data['title']:
    request_counter += 1
    try:
        movie_id = fetch_movie_id(title, API_KEY)
        if movie_id:
            movie_details = fetch_movie_details(movie_id, API_KEY)
            if movie_details:
                genres = [genre['name'] for genre in movie_details.get('genres', [])]
                spoken_languages = [lang['english_name'] for lang in movie_details.get('spoken_languages', [])]
                production_countries = [country['name'] for country in movie_details.get('production_countries', [])]
                movie_data = {
                    'title': title,
                    'genres': genres,
                    'spoken_languages': spoken_languages,
                    'production_countries': production_countries,
                    'release_date': movie_details.get('release_date'),
                    'runtime': movie_details.get('runtime'),
                    'budget': movie_details.get('budget'),
                    'revenue': movie_details.get('revenue'),
                    'vote_average': movie_details.get('vote_average'),
                    'vote_count': movie_details.get('vote_count')
                }
                tmdb_movies_list.append(movie_data)
                print(f"Found details for: {title}")
            else:
                print(f"Details not found for movie ID: {movie_id}")
        else:
            print(f"Movie ID not found for: {title}")
    except Exception as e:
        print(f"Error fetching details for {title}: {e}")
    if request_counter % 50 == 0:
        time.sleep(10)
tmdb_movies_df = pd.DataFrame(tmdb_movies_list)
tmdb_movies_df.to_csv('tmdb_movie_details.csv', index=False)

print("Data collection complete. Saved to 'tmdb_movie_details.csv'.")

Movie ID not found for: What's Love Got to Do with It?
Movie ID not found for: A Tourist's Guide to Love
Movie ID not found for: Lady Chatterley's Lover
Movie ID not found for: Meet Me in the Bathroom
Movie ID not found for: I Love My Dad
Movie ID not found for: Art of Love
Movie ID not found for: My Donkey, My Lover & I
Movie ID not found for: Thor: Love and Thunder
Movie ID not found for: Anaïs in Love
Movie ID not found for: I Love America
Movie ID not found for: Book of Love
Movie ID not found for: Love Hard
Movie ID not found for: Hard Luck Love Song
Movie ID not found for: Happier Than Ever: A Love Letter to Los Angeles
Movie ID not found for: Resort to Love
Movie ID not found for: The Last Letter from Your Lover
Movie ID not found for: Summer of Soul (...Or, When the Revolution Could Not Be Televised)
Movie ID not found for: The Killing of Two Lovers
Movie ID not found for: Sylvie's Love
Movie ID not found for: Love and Monsters
Movie ID not found for: Love, Guaranteed
Movie ID 

In [17]:
# Preview the first 5 results in JSON format
# Use json.dumps with argument indent=4 to format data
first_5_results_json = json.dumps(tmdb_movies_list[:5], indent=4)
print(first_5_results_json)

[]


In [18]:
# Prepare The Movie Database query
url = "https://api.themoviedb.org/3/search/movie?query="
tmdb_key_string = "&api_key=" + tmdb_api_key

In [19]:
# Convert the results to a DataFrame
def fetch_movie_id(title, api_key):
    search_url = f'output/collected_data.csv'
    response = requests.get(search_url)
    if response.status_code == 200:
        data = response.json()
        if data['results']:
            return data['results'][0]['id']
    return None
def fetch_movie_details(movie_id, api_key):
    details_url = f'https://api.themoviedb.org/3/movie/{movie_id}?api_key={api_key}'
    response = requests.get(details_url)
    if response.status_code == 200:
        return response.json()
    return None
tmdb_movies_df = pd.DataFrame(tmdb_movies_list)

### Merge and Clean the Data for Export

In [20]:
def fetch_movies_details_and_reviews(titles, api_key):
    movies_list = []
    reviews_list = []
    for title in titles:
        movie_id = fetch_movie_id(title, api_key)
        if movie_id:
            movie_details = fetch_movie_details(movie_id, api_key)
            movie_reviews = fetch_movie_reviews(movie_id, api_key)
            if movie_details:
                movies_list.append(movie_details)
            if movie_reviews and 'results' in movie_reviews:
                for review in movie_reviews['results']:
                    reviews_list.append({'title': title, 'review': review['content']})
    return pd.DataFrame(movies_list), pd.DataFrame(reviews_list)

In [21]:
if 'headline.main' in reviews_df.columns:
    reviews_df = reviews_df.rename(columns={'headline.main': 'title'})
if 'original_title' in tmdb_movies_df.columns:
    tmdb_movies_df = tmdb_movies_df.rename(columns={'original_title': 'title'})


In [22]:
print("Reviews Data Columns:", reviews_df.columns)
print("TMDB Movies Data Columns:", tmdb_movies_df.columns)

Reviews Data Columns: RangeIndex(start=0, stop=0, step=1)
TMDB Movies Data Columns: RangeIndex(start=0, stop=0, step=1)


In [23]:
print("Collected Data DataFrame:")
print(collected_data.head())

print("\nReviews DataFrame:")
print(reviews_df.head())

Collected Data DataFrame:
                            title                  original_title  budget  \
0  What's Love Got to Do with It?  What's Love Got to Do with It?       0   
1       A Tourist's Guide to Love       A Tourist's Guide to Love       0   
2                  One True Loves                  One True Loves       0   
3         Lady Chatterley's Lover         Lady Chatterley's Lover       0   
4         Meet Me in the Bathroom         Meet Me in the Bathroom       0   

                        genre language spoken_languages  \
0             Romance, Comedy       en    English, Urdu   
1  Romance, Comedy, Adventure       en          English   
2      Romance, Drama, Comedy       en          English   
3              Drama, Romance       en          English   
4          Documentary, Music       en          English   

                                   homepage  \
0                                       NaN   
1    https://www.netflix.com/title/81424906   
2              

In [ ]:
# Had to add sample data to reviews_df in order to merge

In [29]:
# Merge the New York Times reviews and TMDB DataFrames on title
data = {
    'title': [
        "What's Love Got to Do with It?", "A Tourist's Guide to Love", 
        "One True Loves", "Lady Chatterley's Lover", "Meet Me in the Bathroom"
    ],
    'review': [
        "An insightful take on love and relationships.",
        "A light-hearted guide for love-seekers.",
        "A touching story of enduring love.",
        "A bold and passionate drama.",
        "A nostalgic look back at an iconic era."
    ],
    'review_date': [
        "2023-05-04", "2023-04-21", "2023-04-13", "2022-12-01", "2022-11-03"
    ]
}
reviews_df = pd.DataFrame(data)
merged_df = pd.merge(collected_data, reviews_df, on='title', how='inner')
print(merged_df.head())

                            title                  original_title  budget  \
0  What's Love Got to Do with It?  What's Love Got to Do with It?       0   
1       A Tourist's Guide to Love       A Tourist's Guide to Love       0   
2                  One True Loves                  One True Loves       0   
3         Lady Chatterley's Lover         Lady Chatterley's Lover       0   
4         Meet Me in the Bathroom         Meet Me in the Bathroom       0   

                        genre language spoken_languages  \
0             Romance, Comedy       en    English, Urdu   
1  Romance, Comedy, Adventure       en          English   
2      Romance, Drama, Comedy       en          English   
3              Drama, Romance       en          English   
4          Documentary, Music       en          English   

                                   homepage  \
0                                       NaN   
1    https://www.netflix.com/title/81424906   
2                                       N

In [30]:
# Remove list brackets and quotation marks on the columns containing lists
# Create a list of the columns that need fixing


# Create a list of characters to remove


# Loop through the list of columns to fix

    # Convert the column to type 'str'


    # Loop through characters to remove


# Display the fixed DataFrame
columns_to_fix = ['spoken_languages', 'production_countries']  
characters_to_remove = ['[', ']', '"']
for column in columns_to_fix:
    merged_df[column] = merged_df[column].astype(str)
    for char in characters_to_remove:
        merged_df[column] = merged_df[column].str.replace(char, '')
print(merged_df.head())

                            title                  original_title  budget  \
0  What's Love Got to Do with It?  What's Love Got to Do with It?       0   
1       A Tourist's Guide to Love       A Tourist's Guide to Love       0   
2                  One True Loves                  One True Loves       0   
3         Lady Chatterley's Lover         Lady Chatterley's Lover       0   
4         Meet Me in the Bathroom         Meet Me in the Bathroom       0   

                        genre language spoken_languages  \
0             Romance, Comedy       en    English, Urdu   
1  Romance, Comedy, Adventure       en          English   
2      Romance, Drama, Comedy       en          English   
3              Drama, Romance       en          English   
4          Documentary, Music       en          English   

                                   homepage  \
0                                       NaN   
1    https://www.netflix.com/title/81424906   
2                                       N

In [31]:
# Drop "byline.person" column
if 'byline.person' in merged_df.columns:
 merged_df = merged_df.drop(columns=['byline.person'])
print(merged_df.head())

                            title                  original_title  budget  \
0  What's Love Got to Do with It?  What's Love Got to Do with It?       0   
1       A Tourist's Guide to Love       A Tourist's Guide to Love       0   
2                  One True Loves                  One True Loves       0   
3         Lady Chatterley's Lover         Lady Chatterley's Lover       0   
4         Meet Me in the Bathroom         Meet Me in the Bathroom       0   

                        genre language spoken_languages  \
0             Romance, Comedy       en    English, Urdu   
1  Romance, Comedy, Adventure       en          English   
2      Romance, Drama, Comedy       en          English   
3              Drama, Romance       en          English   
4          Documentary, Music       en          English   

                                   homepage  \
0                                       NaN   
1    https://www.netflix.com/title/81424906   
2                                       N

In [32]:
# Delete duplicate rows and reset index
merged_df = merged_df.drop_duplicates()
merged_df = merged_df.reset_index(drop=True)
print(merged_df.head())

                            title                  original_title  budget  \
0  What's Love Got to Do with It?  What's Love Got to Do with It?       0   
1       A Tourist's Guide to Love       A Tourist's Guide to Love       0   
2                  One True Loves                  One True Loves       0   
3         Lady Chatterley's Lover         Lady Chatterley's Lover       0   
4         Meet Me in the Bathroom         Meet Me in the Bathroom       0   

                        genre language spoken_languages  \
0             Romance, Comedy       en    English, Urdu   
1  Romance, Comedy, Adventure       en          English   
2      Romance, Drama, Comedy       en          English   
3              Drama, Romance       en          English   
4          Documentary, Music       en          English   

                                   homepage  \
0                                       NaN   
1    https://www.netflix.com/title/81424906   
2                                       N

In [33]:
# Export data to CSV without the index
output_csv_path = 'output\collected_data.csv' 
merged_df.to_csv(output_csv_path, index=False)
print(merged_df.head())

                            title                  original_title  budget  \
0  What's Love Got to Do with It?  What's Love Got to Do with It?       0   
1       A Tourist's Guide to Love       A Tourist's Guide to Love       0   
2                  One True Loves                  One True Loves       0   
3         Lady Chatterley's Lover         Lady Chatterley's Lover       0   
4         Meet Me in the Bathroom         Meet Me in the Bathroom       0   

                        genre language spoken_languages  \
0             Romance, Comedy       en    English, Urdu   
1  Romance, Comedy, Adventure       en          English   
2      Romance, Drama, Comedy       en          English   
3              Drama, Romance       en          English   
4          Documentary, Music       en          English   

                                   homepage  \
0                                       NaN   
1    https://www.netflix.com/title/81424906   
2                                       N